This notebook is part of [**Byron v0.8**](https://github.com/cad-polito-it/byron)  
Copyright 2023 Giovanni Squillero and Alberto Tonda  
SPDX-License-Identifier: [Apache-2.0](https://www.tldrlegal.com/license/apache-license-2-0-apache-2-0)  

In [34]:
import logging
import random
import pickle
from pprint import pprint
import joblib

## dump/load

In [2]:
beatles = dict()
beatles["John"] = "Lennon"
beatles["Paul"] = "McCartney"
beatles["George"] = "Harrison"
beatles["Ringo"] = "Starr"

In [3]:
try:
    joblib.dump(beatles, "beatles.p")
except OSError as problem:
    print(f"Yeuch: {problem}")

In [4]:
try:
    pprint(joblib.load("beatles.p"))
except OSError as problem:
    print(f"Yeuch: {problem}")

{'George': 'Harrison', 'John': 'Lennon', 'Paul': 'McCartney', 'Ringo': 'Starr'}


In [5]:
with open("beatles.p", "rb") as f:
    pprint(pickle.load(f))

{'George': 'Harrison', 'John': 'Lennon', 'Paul': 'McCartney', 'Ringo': 'Starr'}


## Memory

In [16]:
@joblib.Memory("tmp.dir", verbose=3).cache
def prime(index):
    logging.debug("prime: Calculating %s-nth prime", index)
    primes = [2, 3]
    while len(primes) <= index:
        n = primes[-1] + 2
        while any(n % i == 0 for i in primes):
            n += 2
        primes.append(n)
    return primes[index]

In [17]:
logging.getLogger().setLevel(logging.DEBUG)
prime(10), prime(10), prime(10)

[Memory]0.5s, 0.0min    : Loading prime...
[Memory]0.5s, 0.0min    : Loading prime...
[Memory]0.5s, 0.0min    : Loading prime...


(31, 31, 31)

## Parallel & delayed

In [47]:
def prime(index):
    primes = [2, 3]
    while len(primes) <= index:
        n = primes[-1] + 2
        while any(n % i == 0 for i in primes):
            n += 2
        primes.append(n)
    return primes[index]

In [51]:
random.seed(42)
jobs = list(joblib.delayed(prime)(random.randint(10000, 20000)) for _ in range(20))
jobs

[(<function __main__.prime(index)>, (11824,), {}),
 (<function __main__.prime(index)>, (10409,), {}),
 (<function __main__.prime(index)>, (14506,), {}),
 (<function __main__.prime(index)>, (14012,), {}),
 (<function __main__.prime(index)>, (13657,), {}),
 (<function __main__.prime(index)>, (12286,), {}),
 (<function __main__.prime(index)>, (11679,), {}),
 (<function __main__.prime(index)>, (18935,), {}),
 (<function __main__.prime(index)>, (11424,), {}),
 (<function __main__.prime(index)>, (19674,), {}),
 (<function __main__.prime(index)>, (16912,), {}),
 (<function __main__.prime(index)>, (10520,), {}),
 (<function __main__.prime(index)>, (10488,), {}),
 (<function __main__.prime(index)>, (11535,), {}),
 (<function __main__.prime(index)>, (13582,), {}),
 (<function __main__.prime(index)>, (13811,), {}),
 (<function __main__.prime(index)>, (18279,), {}),
 (<function __main__.prime(index)>, (19863,), {}),
 (<function __main__.prime(index)>, (10434,), {}),
 (<function __main__.prime(inde

In [52]:
%timeit joblib.Parallel(n_jobs=-1)(jobs)

KeyboardInterrupt: 

In [ ]:
%timeit joblib.Parallel(n_jobs=1)(jobs)